### Masking and padding with Keras

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

#### Padding sequence data

处理序列数据的时候，每个样本一般都有不同的长度，如下：

[
  ["The", "weather", "will", "be", "nice", "tomorrow"],
  ["How", "are", "you", "doing", "today"],
  ["Hello", "world", "!"]
]

经过词表 lookup 后，数据可能变成这样：

[
  [83, 91, 1, 645, 1253, 927],
  [73, 8, 3215, 55, 927],
  [71, 1331, 4231]
]

这个 2D list 具有不同的长度，由于深度学习模型的输入数据必须为一个一样的 tensor (比如 shape 为 (batch_size, 6, vocab_size))，所以短句子需要填充，和长句子保持一样(或者把长句子进行裁剪)。

Keras 提供了一个 API，可以方便地进行裁剪和填充：tf.keras.preprocessing.sequence.pad_sequences.

In [2]:
raw_inputs = [
    [83, 91, 1, 645, 1253, 927],
    [73, 8, 3215, 55, 927],
    [711, 632, 71]
]

# 一般是使用 0 作为填充标志
# 可以使用前填充（在头部）和后填充（在尾部），在 RNN 层中推荐使用后填充（为了可以使用 CuDNN 实现）
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                              padding='post')
print(padded_inputs)

[[  83   91    1  645 1253  927]
 [  73    8 3215   55  927    0]
 [ 711  632   71    0    0    0]]


#### Masking

现在所有的样本都有了相同的长度，但必须要告诉模型有些部分实际上是填充的，计算时应该被忽略，这就需要 masking。有三种方式：
1. 增加一个 keras.layers.Masking 层；
2. 配置 keras.layers.Embedding 层为 mask_zero=True;
3. 调用层时手动传一个 mask 参数。

#### Mask-generating layers: Embedding and Masking

In [4]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)
print(masked_output._keras_mask)

tf.Tensor(
[[ True  True  True  True  True  True]
 [ True  True  True  True  True False]
 [ True  True  True False False False]], shape=(3, 6), dtype=bool)


In [8]:
masking_layer = layers.Masking()
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]),
    tf.float32)
masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True  True  True  True]
 [ True  True  True  True  True False]
 [ True  True  True False False False]], shape=(3, 6), dtype=bool)


#### Mask propagation in the Functional API and Sequential API

当使用 Functional 或 Sequential API 时，Embedding 或 Masking 层产生的 mask 可以在网络任意层中传播。Keras 可以自动获取 input 对应的 mask。但在 subclassed 的模型或层中的 call 方法中，mask 不会自动传播，必须手动传 mask 参数。在下面的 Sequential 模型中，LSTM 层可以自动取到 mask，并忽略掉填充的值。

In [9]:
model = tf.keras.Sequential([
    layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True),
    layers.LSTM(32)
])

下面的 Functional API 模型也是如此：

In [10]:
inputs = tf.keras.Input(shape=(None,), dtype='int32')
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = tf.keras.Model(inputs, outputs)

#### Passing mask tensors directly to layers

可以处理 masks（比如 LSTM）的层在 __call__ 方法中都有 mask 参数，同时，产生 mask 的层（比如 Embedding）都有 compute_mask(input, previous_mask) 可以调用。

In [11]:
class MyLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
        self.lstm = layers.LSTM(32)
        
    def call(self, inputs):
        x = self.embedding(inputs)
        mask = self.embedding.compute_mask(inputs)
        output = self.lstm(x, mask=mask)
        return output
    
layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype('int32')
layer(x)

<tf.Tensor: id=4750, shape=(32, 32), dtype=float32, numpy=
array([[-5.4335361e-04,  4.5744404e-03, -2.5518630e-03, ...,
        -1.2719275e-03,  4.5639314e-04,  1.7747076e-02],
       [ 4.0839505e-03,  2.5836329e-04, -1.4106500e-03, ...,
         6.3203569e-03, -3.2956379e-03,  6.1929859e-03],
       [-3.7946794e-03, -4.6657557e-03, -3.6887231e-03, ...,
         2.7973948e-03, -1.7516615e-03,  3.3951143e-03],
       ...,
       [ 4.0407674e-03,  1.0803455e-03, -5.1380898e-04, ...,
         3.5285761e-04, -5.5053877e-03,  4.0466321e-04],
       [ 1.7662996e-03,  5.3380225e-03,  4.0492108e-03, ...,
         1.0195142e-03, -3.0410581e-03, -1.4063568e-03],
       [ 8.2265725e-03,  9.9298632e-06, -7.7274878e-04, ...,
        -3.0296063e-03, -1.0024327e-02,  3.2026954e-03]], dtype=float32)>

#### Supporting masking in your custom layers

有时候需要写生成 mask 的层（如 Embedding），或者需要修改当前 mask 的层。为了实现这个功能，层需要实现 layer.compute_mask() 方法，其根据输入和当前 mask 生成新的 mask。例子如下：

In [18]:
class TemporalSplit(tf.keras.layers.Layer):
    """Split the input tensor into 2 tensors along the time dimension."""
    
    def call(self, inputs):
        # Expect the input to be 3D and mask to be 2D, split the input tensor into 2
        # subtensors along the time axis (axis 1).
        return tf.split(inputs, 2, axis=1)
    
    def compute_mask(self, inputs, mask=None):
        # Also split the mask into 2 if it presents.
        if mask is None:
            return None
        return tf.split(mask, 2, axis=1)
    
first_half, second_half = TemporalSplit()(masked_embedding)
print(masked_embedding._keras_mask)
print(first_half._keras_mask)
print(second_half._keras_mask)

tf.Tensor(
[[ True  True  True  True  True  True]
 [ True  True  True  True  True False]
 [ True  True  True False False False]], shape=(3, 6), dtype=bool)
tf.Tensor(
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]], shape=(3, 3), dtype=bool)
tf.Tensor(
[[ True  True  True]
 [ True  True False]
 [False False False]], shape=(3, 3), dtype=bool)


In [12]:
a = tf.constant([[[1, 2, 3],
                  [4, 5, 6]],
                 [[7, 8, 9],
                  [10, 11, 12]]]) # (2, 2, 3)

b = tf.split(a, 2, axis=1)
b

[<tf.Tensor: id=4757, shape=(2, 1, 3), dtype=int32, numpy=
 array([[[1, 2, 3]],
 
        [[7, 8, 9]]])>,
 <tf.Tensor: id=4758, shape=(2, 1, 3), dtype=int32, numpy=
 array([[[ 4,  5,  6]],
 
        [[10, 11, 12]]])>]

另一个例子：

In [20]:
class CustomEmbedding(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
        super(CustomEmbedding, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.mask_zero = mask_zero
        
    def build(self, input_shape):
        self.embeddings = self.add_weight(
            shape=(self.input_dim, self.output_dim),
            initializer='random_normal',
            dtype='float32')
        
    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)
    
    def compute_mask(self, inputs, mask=None):
        if not self.mask_zero:
            return None
        return tf.not_equal(inputs, 0)
    
layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype('int32')

y = layer(x)
mask = layer.compute_mask(x)

print(mask)

tf.Tensor(
[[False  True  True  True  True  True False  True  True  True]
 [ True False  True  True  True  True  True False  True  True]
 [ True  True  True  True  True False False  True  True  True]], shape=(3, 10), dtype=bool)


#### Writing layers that need mask information

有些层需要根据 mask 做出判断，在 call 中接受 mask 参数，来判断是否跳过某些时间步。可以在 call 中加入 mask=None 参数。

In [ ]:
class MaskConsumer(tf.keras.layers.Layer):
    def call(self, inputs, mask=None):
        ...